In [68]:
import pandas as pd
import numpy as np
from gensim import corpora, models
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse.linalg import svds
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import word2vec, Phrases
from gensim.models.phrases import Phraser

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import nltk

In [17]:
df = pd.read_csv("mydata.csv")
df = df.drop(["Unnamed: 0"],axis = 1)

In [18]:
df.head()

Speaker                                           FileName  \
0    BUSH  ../Project/Data/BUSH/100th Anniversary of Cuba...   
1    BUSH  ../Project/Data/BUSH/2004 Presidential Electio...   
2    BUSH  ../Project/Data/BUSH/2004 Republican National ...   
3    BUSH  ../Project/Data/BUSH/2008 Republican National ...   
4    BUSH  ../Project/Data/BUSH/911 Address to the Nation...   

                                             RawText  \
0   thank you very much. sientese. voy a hablar e...   
1   thank you all. thank you all for coming. we h...   
2   mr. chairman! mr. chairman, delegates, fellow...   
3   thank you, laura, and good evening. as you kn...   
4   good evening.  today, our fellow citizens, ou...   

                                               Token  \
0  ['thank', 'much', 'sientese', 'voy', 'hablar',...   
1  ['thank', 'thank', 'coming', 'long', 'night', ...   
2  ['chairman', 'chairman', 'delegates', 'fellow'...   
3  ['thank', 'laura', 'good', 'evening', 'know', ...   
4  ['good', 'evening', 'today', 'fellow', 'citize...   

                                        Cleaned_text  \
0  thank much sientese voy hablar en espanol hoy ...   
1  thank thank coming long night great night vote...   
2  chairman chairman delegates fellow citizens ho...   
3  thank laura good evening know duties washingto...   
4  good evening today fellow citizens way life fr...   

                                          Sent_Token  \
0  [' thank you very much.', 'sientese.', 'voy a ...   
1  [' thank you all.', 'thank you all for coming....   
2  [' mr. chairman!', 'mr. chairman, delegates, f...   
3  [' thank you, laura, and good evening.', 'as y...   
4  [' good evening.', 'today, our fellow citizens...   

                                     Normalized_list  
0  [['thank', 'much'], ['sientese'], ['voy', 'hab...  
1  [['thank'], ['thank'], ['long', 'night', 'grea...  
2  [['chairman'], ['chairman', 'delegate', 'fello...  
3  [['thank', 'laura', 'good', 'evening'], ['know...  
4  [['good', 'evening'], ['today', 'fellow', 'cit...

In [4]:
def remove_punctuation(l):
    r = []
    not_wanted = ["‘","’",'”','“','–']
    for i in l:
        removed = [x for x in i if x not in not_wanted]
        r.append(removed)
    return r

# Topic modeling for two parties using LSI
Rep = ["TRUMP","BUSH",]
Dem = ["HILLARY","OBAMA","SANDERS"]

Rep_df = df.loc[df['Speaker'].isin(Rep)]
Dem_df = df.loc[df['Speaker'].isin(Dem)]

In [5]:
dic_party_num = {}
dic_party_num["Republican"] = Rep_df.shape[0]
dic_party_num["Democratic"] = Dem_df.shape[0]

### Topic modeling for two parties using Tf-idf and LSI

In [22]:
Rep_col2list = list(Rep_df.Normalized_list.apply(literal_eval))
Rep_flat_list = [item for sublist in Rep_col2list for item in sublist]
Rep_flat_list = remove_punctuation(Rep_flat_list)

Dem_col2list = list(Dem_df.Normalized_list.apply(literal_eval))
Dem_flat_list = [item for sublist in Dem_col2list for item in sublist]
Dem_flat_list = remove_punctuation(Dem_flat_list)

Rep_dictionary = corpora.Dictionary(Rep_flat_list)
Dem_dictionary = corpora.Dictionary(Dem_flat_list)

Rep_corpus = [Rep_dictionary.doc2bow(text) for text in Rep_flat_list]
Dem_corpus = [Dem_dictionary.doc2bow(text) for text in Dem_flat_list]

Rep_tfidf = models.TfidfModel(Rep_corpus)
Rep_corpus_tfidf = Rep_tfidf[Rep_corpus]
Dem_tfidf = models.TfidfModel(Dem_corpus)
Dem_corpus_tfidf = Dem_tfidf[Dem_corpus]

total_topics = 2
Rep_lsi = models.LsiModel(Rep_corpus_tfidf,
id2word= Rep_dictionary,
num_topics=total_topics)
Dem_lsi = models.LsiModel(Dem_corpus_tfidf,
id2word= Dem_dictionary,
num_topics=total_topics)

def print_topics_gensim(topic_model, total_topics=1,
    weight_threshold=0.0001,
    display_weights=False,
    num_terms=None):
    
    for index in range(total_topics):
        topic = topic_model.show_topic(index)
        topic = [(word, round(wt,2)) for word, wt in topic if abs(wt) >= weight_threshold]
        if display_weights:
            print('Topic #'+str(index+1)+' with weights')
            print(topic[:num_terms] if num_terms else topic)
        else:
            print('Topic #'+str(index+1)+' without weights')
            tw = [term for term, wt in topic]
            print(tw[:num_terms] if num_terms else tw)
    print("")
print("Topics for Republicans:")
print_topics_gensim(topic_model=Rep_lsi, total_topics=total_topics, num_terms=10, display_weights=False)
print("Topics for Democrats:")
print_topics_gensim(topic_model=Dem_lsi, total_topics=total_topics, num_terms=10, display_weights=False)

Topics for Republicans:
Topic #1 without weights
['know', 'people', 'right', 'want', 'happen', 'great', 'think', 'country', 'like', 'thing']
Topic #2 without weights
['right', 'know', 'people', 'great', 'country', 'want', 'thing', 'think', 'good', 'america']

Topics for Democrats:
Topic #1 without weights
['know', 'right', 'people', 'work', 'want', 'country', 'think', 'american', 'america', 'us']
Topic #2 without weights
['right', 'know', 'applause', 'want', 'well', 'country', 'american', 'let', 'need', 'people']



### build the term document tf-idf weighted matrix

In [31]:
def build_feature_matrix(documents, feature_type='frequency'):
    feature_type = feature_type.lower().strip()  
    if feature_type == 'binary':
        vectorizer = CountVectorizer(binary=True, min_df=1, 
                                     ngram_range=(1, 1))
    elif feature_type == 'frequency':
        vectorizer = CountVectorizer(binary=False, min_df=1, 
                                     ngram_range=(1, 1))
    elif feature_type == 'tfidf':
        vectorizer = TfidfVectorizer(min_df=1, 
                                     ngram_range=(1, 1))
    else:
        raise Exception("Wrong feature type entered. Possible values: 'binary', 'frequency', 'tfidf'")
    feature_matrix = vectorizer.fit_transform(documents).astype(float)    
    return vectorizer, feature_matrix

def low_rank_svd(matrix, singular_count=2):    
    u, s, vt = svds(matrix, k=singular_count)
    return u, s, vt

def get_topics_terms_weights(weights, feature_names):
    feature_names = np.array(feature_names)
    sorted_indices = np.array([list(row[::-1]) for row in np.argsort(np.abs(weights))]) 
    sorted_weights = np.array([list(wt[index]) for wt, index in zip(weights,sorted_indices)])
    sorted_terms = np.array([list(feature_names[row]) for row in sorted_indices])
    topics = [np.vstack((terms.T, term_weights.T)).T for terms, term_weights in zip(sorted_terms, sorted_weights)]
    return topics

def print_topics_udf(topics, total_topics=1, weight_threshold=0.0001, display_weights=False, num_terms=None):
    for index in range(total_topics):
        topic = topics[index]
        topic = [(term, float(wt)) for term, wt in topic]
        topic = [(word, round(wt,2)) for word, wt in topic if abs(wt) >= weight_threshold]
        if display_weights:
            print('Topic #'+str(index+1)+' with weights')
            print(topic[:num_terms] if num_terms else topic)
        else:
            print('Topic #'+str(index+1)+' without weights')
            tw = [term for term, wt in topic]
            print(tw[:num_terms] if num_terms else tw)
        print("")

In [38]:
# Topic modeling for two parties using SVD and LSI
# build the term document tf-idf weighted matrix

Rep_df_all_sentence_list = []
for i in Rep_flat_list:
    Rep_df_all_sentence_list.append(" ".join(i))
    
Dem_df_all_sentence_list = []
for i in Dem_flat_list:
    Dem_df_all_sentence_list.append(" ".join(i))

Rep_vectorizer, Rep_tfidf_matrix = build_feature_matrix(Rep_df_all_sentence_list,
feature_type='tfidf')
Rep_td_matrix = Rep_tfidf_matrix.transpose()
Rep_td_matrix = Rep_td_matrix.multiply(Rep_td_matrix > 0)

Dem_vectorizer, Dem_tfidf_matrix = build_feature_matrix(Dem_df_all_sentence_list,
feature_type='tfidf')
Dem_td_matrix = Dem_tfidf_matrix.transpose()
Dem_td_matrix = Dem_td_matrix.multiply(Dem_td_matrix > 0)


# fix total topics and get the terms used in the term-document matrix
total_topics = 2
Rep_feature_names = Rep_vectorizer.get_feature_names()
Dem_feature_names = Dem_vectorizer.get_feature_names()

u1, s1, vt1 = low_rank_svd(Rep_td_matrix, singular_count=total_topics)
weights1 = u1.transpose() * s1[:, None]

u2, s2, vt2 = low_rank_svd(Dem_td_matrix, singular_count=total_topics)
weights2 = u2.transpose() * s2[:, None]

topics1 = get_topics_terms_weights(weights1, Rep_feature_names)
topics2 = get_topics_terms_weights(weights2, Dem_feature_names)

print("For Republican parties, topic words are:")
print_topics_udf(topics=topics1,total_topics=total_topics, weight_threshold=1,display_weights=False)
print("")
print("For Democratic parties, topic words are:")
print_topics_udf(topics=topics2,total_topics=total_topics, weight_threshold=1,display_weights=False)

For Republican parties, topic words are:
Topic #1 without weights
['right', 'know', 'people', 'great', 'think', 'want', 'country', 'thing']

Topic #2 without weights
['know', 'people', 'right', 'want', 'great', 'think', 'happen', 'country', 'like', 'thing']


For Democratic parties, topic words are:
Topic #1 without weights
['know', 'right', 'people', 'work', 'country', 'american', 'us', 'america', 'think', 'need', 'job']

Topic #2 without weights
['know', 'people', 'work', 'want', 'right', 'country', 'think', 'american', 'america', 'us', 'need', 'well', 'good', 'believe', 'family', 'job', 'let', 'president', 'state', 'together', 'year', 'every', 'take', 'time', 'lot', 'thing', 'vote', 'hard', 'many', 'care', 'talk', 'much', 'woman', 'help', 'pay', 'like', 'way', 'would', 'million', 'world', 'campaign', 'change', 'issue', 'young', 'economy', 'happen', 'today', 'really', 'life', 'health', 'new', 'great', 'united', 'look', 'give', 'child', 'stand', 'mean', 'everybody']



###  train a NN (neural network) Word2Vec model

In [49]:
# Rep_df.Normalized_list = Rep_df.Normalized_list.apply(literal_eval)
Dem_df.Normalized_list = Dem_df.Normalized_list.apply(literal_eval)
Dem_sent = Dem_df.Normalized_list.sum()
Dem_sent = remove_punctuation(Dem_sent)

In [50]:
# Create Dictionary
Dem_id2word = corpora.Dictionary(Dem_sent)

# Create Corpus: Term Document Frequency
Dem_corpus = [Dem_id2word.doc2bow(text) for text in Dem_sent]

In [53]:
%%time
Dem_lda_model = gensim.models.ldamodel.LdaModel(corpus=Dem_corpus,
                                           id2word=Dem_id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='auto',
                                           iterations=100,
                                           per_word_topics=True)

Wall time: 13min 44s


In [56]:
# Print the Keyword in the 10 topics for Democratic party
pprint(Dem_lda_model.print_topics())
Dem_doc_lda = Dem_lda_model[Dem_corpus]

[(0,
  '0.148*"wall" + 0.105*"order" + 0.079*"break" + 0.063*"senator" + '
  '0.058*"meeting" + 0.041*"everyone" + 0.037*"room" + 0.021*"depression" + '
  '0.018*"crash" + 0.011*"defense"'),
 (1,
  '0.089*"million" + 0.063*"many" + 0.049*"job" + 0.038*"talk" + 0.030*"term" '
  '+ 0.029*"climate" + 0.025*"new" + 0.023*"create" + 0.022*"bill" + '
  '0.021*"second"'),
 (2,
  '0.000*"capandtrade" + 0.000*"marketbased" + 0.000*"underpin" + '
  '0.000*"uphold" + 0.000*"cyberthreats" + 0.000*"espionage" + '
  '0.000*"confidential" + 0.000*"cyberenabled" + 0.000*"knowingly" + '
  '0.000*"theft"'),
 (3,
  '0.045*"know" + 0.031*"campaign" + 0.025*"care" + 0.021*"let" + '
  '0.021*"health" + 0.020*"republican" + 0.019*"year" + 0.018*"change" + '
  '0.018*"end" + 0.017*"take"'),
 (4,
  '0.000*"capandtrade" + 0.000*"marketbased" + 0.000*"underpin" + '
  '0.000*"uphold" + 0.000*"cyberthreats" + 0.000*"espionage" + '
  '0.000*"confidential" + 0.000*"cyberenabled" + 0.000*"knowingly" + '
  '0.000*"the

In [58]:
# Rep_df.Normalized_list = Rep_df.Normalized_list.apply(literal_eval)
Rep_sent = Rep_df.Normalized_list.sum()
Rep_sent = remove_punctuation(Rep_sent)

In [59]:
# Create Dictionary
Rep_id2word = corpora.Dictionary(Rep_sent)

# Create Corpus: Term Document Frequency
Rep_corpus = [Rep_id2word.doc2bow(text) for text in Rep_sent]

In [60]:
%%time
Rep_lda_model = gensim.models.ldamodel.LdaModel(corpus=Rep_corpus,
                                           id2word=Rep_id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='auto',
                                           iterations=100,
                                           per_word_topics=True)

Wall time: 16min 26s


In [61]:
# Print the Keyword in the 10 topics for Republican party
pprint(Rep_lda_model.print_topics())
Rep_doc_lda = Rep_lda_model[Rep_corpus]

[(0,
  '0.000*"explicitly" + 0.000*"fiveyear" + 0.000*"experienced" + '
  '0.000*"prosecution" + 0.000*"accuse" + 0.000*"bodyguard" + 0.000*"hamdan" + '
  '0.000*"legality" + 0.000*"circuit" + 0.000*"corroborate"'),
 (1,
  '0.065*"know" + 0.054*"want" + 0.032*"clinton" + 0.032*"much" + '
  '0.023*"happen" + 0.019*"let" + 0.016*"incredible" + 0.016*"support" + '
  '0.016*"president" + 0.016*"world"'),
 (2,
  '0.047*"people" + 0.032*"great" + 0.032*"country" + 0.015*"american" + '
  '0.014*"many" + 0.014*"like" + 0.013*"time" + 0.013*"immigration" + '
  '0.013*"take" + 0.012*"really"'),
 (3,
  '0.000*"explicitly" + 0.000*"fiveyear" + 0.000*"experienced" + '
  '0.000*"prosecution" + 0.000*"accuse" + 0.000*"bodyguard" + 0.000*"hamdan" + '
  '0.000*"legality" + 0.000*"circuit" + 0.000*"corroborate"'),
 (4,
  '0.000*"explicitly" + 0.000*"fiveyear" + 0.000*"experienced" + '
  '0.000*"prosecution" + 0.000*"accuse" + 0.000*"bodyguard" + 0.000*"hamdan" + '
  '0.000*"legality" + 0.000*"circuit" +

In [74]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(Dem_lda_model, Dem_corpus, dictionary=Dem_lda_model.id2word, mds='mmds')

In [75]:
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
9      38.565563        1       1  0.146847 -0.420504
3      27.263508        1       2 -0.194458 -0.386614
6      12.867351        1       3 -0.419088 -0.127074
8      10.996427        1       4 -0.372831  0.283180
1       9.359102        1       5  0.439793 -0.142084
0       0.888507        1       6  0.216385  0.263942
7       0.014884        1       7  0.045838  0.132289
2       0.014884        1       8  0.045838  0.132289
5       0.014884        1       9  0.045838  0.132289
4       0.014884        1      10  0.045838  0.132289, topic_info=      Category         Freq          Term        Total  loglift  logprob
term                                                                    
28     Default  8747.000000        people  8747.000000  30.0000  30.0000
96     Default  3815.000000         think  3815.000000  29.0000  29.0000
214    Default  3744.000000       country  3744.000000  28.0000  28.0000
108    Default  2546.000000       million  2546.000000  27.0000  27.0000
4      Default  3737.000000          know  3737.000000  26.0000  26.0000
158    Default  1803.000000          many  1803.000000  25.0000  25.0000
478    Default  1815.000000         state  1815.000000  24.0000  24.0000
85     Default  1526.000000          time  1526.000000  23.0000  23.0000
366    Default  2567.000000      campaign  2567.000000  22.0000  22.0000
1173   Default  1633.000000         trump  1633.000000  21.0000  21.0000
26     Default  1412.000000           job  1412.000000  20.0000  20.0000
397    Default  1439.000000         would  1439.000000  19.0000  19.0000
145    Default  1256.000000          need  1256.000000  18.0000  18.0000
125    Default  2113.000000          care  2113.000000  17.0000  17.0000
175    Default  1080.000000          talk  1080.000000  16.0000  16.0000
1219   Default  1113.000000         party  1113.000000  15.0000  15.0000
0      Default  2518.000000      american  2518.000000  14.0000  14.0000
90     Default  1076.000000    democratic  1076.000000  13.0000  13.0000
334    Default  2444.000000          want  2444.000000  12.0000  12.0000
64     Default  1768.000000           let  1768.000000  11.0000  11.0000
164    Default  1124.000000           lot  1124.000000  10.0000  10.0000
78     Default  1032.000000         every  1032.000000   9.0000   9.0000
126    Default  1734.000000        health  1734.000000   8.0000   8.0000
65     Default  1667.000000    republican  1667.000000   7.0000   7.0000
212    Default   990.000000          life   990.000000   6.0000   6.0000
71     Default  1584.000000          year  1584.000000   5.0000   5.0000
141    Default   869.000000          term   869.000000   4.0000   4.0000
167    Default   996.000000          wage   996.000000   3.0000   3.0000
280    Default   921.000000         money   921.000000   2.0000   2.0000
232    Default  1538.000000        change  1538.000000   1.0000   1.0000
...        ...          ...           ...          ...      ...      ...
11822  Topic10     0.003846  islamophobia     0.892005   3.3663  -9.3778
3937   Topic10     0.003846    accelerate     0.628720   3.7161  -9.3778
3946   Topic10     0.003846          rich   175.040375  -1.9130  -9.3778
3945   Topic10     0.003846           gut   141.047882  -1.6971  -9.3778
3944   Topic10     0.003846          wing     4.971260   1.6483  -9.3778
3943   Topic10     0.003846         hobby     0.628634   3.7162  -9.3778
3942   Topic10     0.003846        review     0.629568   3.7147  -9.3778
3941   Topic10     0.003846        ripple     0.628635   3.7162  -9.3778
3940   Topic10     0.003846   restriction     1.856113   2.6335  -9.3778
3939   Topic10     0.003846       expanse     0.628634   3.7162  -9.3778
3938   Topic10     0.003846            54     0.670430   3.6518  -9.3778
3936   Topic10     0.003846       shudder     0.628634   3.7162  -9.3778
3879   Topic10     0.003

In [72]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(Rep_lda_model, Rep_corpus, dictionary=Rep_lda_model.id2word, mds='mmds')

In [73]:
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      50.661541        1       1 -0.415677  0.179112
7      25.739479        1       2 -0.045036 -0.453174
1      22.322998        1       3 -0.372183 -0.206782
9       1.248698        1       4  0.014909  0.330697
0       0.004648        1       5  0.136331  0.025024
4       0.004618        1       6  0.136331  0.025024
8       0.004506        1       7  0.136331  0.025024
6       0.004506        1       8  0.136331  0.025024
3       0.004506        1       9  0.136331  0.025024
5       0.004506        1      10  0.136331  0.025024, topic_info=      Category         Freq              Term        Total  loglift  logprob
term                                                                        
393    Default  3378.000000              know  3378.000000  30.0000  30.0000
60     Default  2823.000000              want  2823.000000  29.0000  29.0000
32     Default  5509.000000            people  5509.000000  28.0000  28.0000
101    Default  2235.000000               job  2235.000000  27.0000  27.0000
3716   Default   642.000000           illegal   642.000000  26.0000  26.0000
84     Default  3805.000000             great  3805.000000  25.0000  25.0000
166    Default  3722.000000           country  3722.000000  24.0000  24.0000
1092   Default  1658.000000           clinton  1658.000000  23.0000  23.0000
0      Default  1640.000000              much  1640.000000  22.0000  22.0000
351    Default  1654.000000             right  1654.000000  21.0000  21.0000
455    Default  1578.000000              work  1578.000000  20.0000  20.0000
364    Default  1498.000000               way  1498.000000  19.0000  19.0000
129    Default  1426.000000                us  1426.000000  18.0000  18.0000
516    Default  1212.000000            happen  1212.000000  17.0000  17.0000
372    Default  1183.000000               new  1183.000000  16.0000  16.0000
95     Default  1154.000000             state  1154.000000  15.0000  15.0000
394    Default  1005.000000               let  1005.000000  14.0000  14.0000
38     Default  1076.000000         important  1076.000000  13.0000  13.0000
432    Default  1029.000000               law  1029.000000  12.0000  12.0000
78     Default  1785.000000          american  1785.000000  11.0000  11.0000
1906   Default   973.000000            border   973.000000  10.0000  10.0000
22     Default  1691.000000              many  1691.000000   9.0000   9.0000
635    Default   845.000000        incredible   845.000000   8.0000   8.0000
353    Default   831.000000           support   831.000000   7.0000   7.0000
327    Default   903.000000           amazing   903.000000   6.0000   6.0000
169    Default  1617.000000              like  1617.000000   5.0000   5.0000
531    Default   820.000000         president   820.000000   4.0000   4.0000
52     Default   808.000000             world   808.000000   3.0000   3.0000
116    Default  1565.000000              time  1565.000000   2.0000   2.0000
680    Default   796.000000          together   796.000000   1.0000   1.0000
...        ...          ...               ...          ...      ...      ...
10871  Topic10     0.000965  underappreciated     0.852179   3.2236  -9.2939
3620   Topic10     0.000965            rocket     0.472248   3.8139  -9.2939
3628   Topic10     0.000965          lynchpin     0.472248   3.8139  -9.2939
3627   Topic10     0.000965        impossible     1.160193   2.9151  -9.2939
3626   Topic10     0.000965              asia     0.517871   3.7217  -9.2939
3625   Topic10     0.000965             total   242.360245  -2.4268  -9.2939
3624   Topic10     0.000965          genocide     0.500127   3.7566  -9.2939
3623   Topic10     0.000965              bold   101.740440  -1.5588  -9.2939
3622   Topic10     0.000965             herzl     0.472248   3.8139  -9.2939
3621   Topic10     0.000965              1948     0.472248   3.8139  -9.293